In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
from datetime import datetime
#import sqlalchemy as sa

In [2]:
sys.path.append(r"../../src/collectcube")
from db import *
from db_post import *

In [3]:
#out_dir = '../../data/LUCinLA_paraguay/smSamp'
#local_db_path = os.path.join(out_dir, 'landcover.db')
out_dir = 'C:/GISprojects/ParaguayValidation/smDistricts'
local_db_path = 'C:/GISprojects/ParaguayValidation/smDistricts/landcover.db'

In [4]:
table_check = check_table(local_db_path,'pixels')

             PID  Center     cent_X     cent_Y   cent_lat  cent_long  ransamp  \
71995  0007999_4       0  3255940.0 -3073290.0 -24.288644 -54.514667        1   
71996  0007999_5       0  3255960.0 -3073290.0 -24.288644 -54.514449        1   
71997  0007999_6       0  3255940.0 -3073300.0 -24.288725 -54.514656        1   
71998  0007999_7       0  3255950.0 -3073300.0 -24.288725 -54.514547        1   
71999  0007999_8       0  3255960.0 -3073300.0 -24.288725 -54.514438        1   

       checked  PID0  PID1           sampgroup  
71995        0  7999     4  D1409_Canindeyu_LP  
71996        0  7999     5  D1409_Canindeyu_LP  
71997        0  7999     6  D1409_Canindeyu_LP  
71998        0  7999     7  D1409_Canindeyu_LP  
71999        0  7999     8  D1409_Canindeyu_LP  


## Data cleaning (e.g. if pandas import below doesn't work)
Note: need for this should be eliminated as more formatting restrictions are placed in db entry functions

### manipulating table (should not need to do this here)

In [ ]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
## adding a new column to existing table
c.execute("ALTER TABLE PixelVerification ADD COLUMN entry_lev INTEGER")
## filling column in existing table
#c.execute("UPDATE PixelVerification SET entry_lev = 4 WHERE entry_lev IS NULL")
con.commit()      
c.close()

### querying table (e.g. for bad records)

In [5]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("SELECT * FROM PixelVerification  WHERE imgDate = '2021_03-07'")
con.commit()
print(c.fetchone()) 
'''

con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("SELECT * FROM pixels WHERE PID0 = '500'")
con.commit()
print(c.fetchone()) 

('0000500_0', 1, 3073210.0, -2919110.0, -23.03376311127423, -56.65315280342373, 1, 0, 500, 0, 'D0112_Concepcion_A')


### fixing bad records

In [10]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE PixelVerification SET imgDate = '2022-01-01' WHERE imgDate ='1/1/2022'")
con.commit()
c.close()
'''

'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE pixels SET sampgroup = 'D0112_Concepcion_A' WHERE PID0 ='998'")
con.commit()
c.close()
'''

'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE PixelVerification SET PID0 = '4312' WHERE recID = '8588'")
con.commit()
c.close()
'''
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("DELETE FROM PixelVerification WHERE recID >= '12290'")
con.commit()
c.close()
'''

'\ncon = sqlite3.connect(local_db_path)\nc = con.cursor()\nc.execute("DELETE FROM PixelVerification WHERE recID >= \'12290\'")\ncon.commit()\nc.close()\n'

## Open db table as pandas dataframe

In [5]:
## This uses sqlalchemy but is prefered because it will make sure that formatting is correct
df = open_data_records_sqlachemy(local_db_path)
df.tail()

,recID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,PercentTree,...,TREEPLANT0,HIGHVEG,TREEPLANT,FOREST,Age,Stability,State,Notes,entry_lev,PIDi
11924,13794,5794,0,2008-10-12,20.0,12.0,8.0,,,,...,,,,,,--,--,,1,0005794_0
11925,13795,5794,0,2018-03-17,20.0,12.0,8.0,,,,...,,,,,,--,--,,1,0005794_0
11926,13796,7321,0,2023-03-22,20.0,13.0,8.0,,,,...,,,,,,--,--,,1,0007321_0
11927,13797,5841,0,2023-03-20,20.0,35.0,8.0,,,,...,,,,,,--,--,,1,0005841_0
11928,13798,5841,0,2013-07-10,20.0,12.0,8.0,,,,...,,,,,,--,--,,1,0005841_0


In [5]:
## This uses only sqllite and can be used to examine table if the above fails
conn = sqlite3.connect(local_db_path, isolation_level=None,
    detect_types=sqlite3.PARSE_COLNAMES)
db_df = pd.read_sql_query("SELECT * FROM PixelVerification", conn)
db_df.tail()
#db_df.to_csv(os.path.join('C:/GISprojects/ParaguayValidation/smDistricts','sm_test_db.csv'), index=False)

,recID,PID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,...,MEDVEG,TREEPLANT0,HIGHVEG,TREEPLANT,FOREST,Age,Stability,State,Notes,entry_lev
11924,13794,5794_0,5794,0,2008-10-12,20.0,12.0,8.0,,,...,,,,,,,--,--,,1
11925,13795,5794_0,5794,0,2018-03-17,20.0,12.0,8.0,,,...,,,,,,,--,--,,1
11926,13796,7321_0,7321,0,2023-03-22,20.0,13.0,8.0,,,...,,,,,,,--,--,,1
11927,13797,5841_0,5841,0,2023-03-20,20.0,35.0,8.0,,,...,,,,,,,--,--,,1
11928,13798,5841_0,5841,0,2013-07-10,20.0,12.0,8.0,,,...,,,,,,,--,--,,1


In [8]:
#addfile = 'C:/GISprojects/ParaguayValidation/lgDistricts/lg_addF.csv'
#pd.read_csv(addfile).to_sql('PixelVerification', conn, if_exists='append', index=False)

1476

## Get sample for target date (direct and indirect)

In [6]:
## Will consider anything 1-June of prior year to 1-June of target_date year as direct observation 
## and interpolate from other observations according to the rules set up

target_date = 2022

sample_type = 'area_est' ## 'area_est',  validation', 'training' 
sample_cats = 'bi_crop'   ## 'bi_tree', 'bi_forest'
lut = '../../data/Class_LUT.csv'
   
all_obs=get_sample_for_date(target_date,local_db_path,sample_type,sample_cats,lut)
    
all_obs.sort_values('PID0')
all_obs.tail(n=10)
#all_obs.to_csv(os.path.join(out_dir,'samp2022.csv'))

there are 6031 observations for selected date
there are 5823 center pixel observations


,recID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,PercentTree,...,TREEPLANT,FOREST,Age,Stability,State,Notes,entry_lev,PIDi,obs_type,LC_UNQ
11886,13756.0,7990,0,2022-01-01,20.0,30.0,NaN,None,None,None,...,None,None,None,None,None,None,1,0007990_0,direct_GE,31.0
11887,13757.0,7991,0,2022-01-01,70.0,80.0,NaN,None,None,None,...,None,None,None,None,None,None,1,0007991_0,direct_GE,80.0
11888,13758.0,7992,0,2022-01-01,20.0,30.0,NaN,None,None,None,...,None,None,None,None,None,None,1,0007992_0,direct_GE,31.0
11889,13759.0,7993,0,2022-01-01,20.0,30.0,NaN,None,None,None,...,None,None,None,None,None,None,1,0007993_0,direct_GE,31.0
11890,13760.0,7994,0,2022-01-01,70.0,80.0,NaN,None,None,None,...,None,None,None,None,None,None,1,0007994_0,direct_GE,80.0
11891,13761.0,7995,0,2022-01-01,20.0,30.0,NaN,None,None,None,...,None,None,None,None,None,None,1,0007995_0,direct_GE,30.0
11892,13762.0,7996,0,2022-01-01,20.0,98.0,NaN,None,None,None,...,None,None,None,None,None,None,1,0007996_0,direct_GE,13.0
11893,13763.0,7997,0,2022-01-01,20.0,98.0,NaN,None,None,None,...,None,None,None,None,None,None,1,0007997_0,direct_GE,17.0
11894,13764.0,7998,0,2022-01-01,20.0,30.0,NaN,None,None,None,...,None,None,None,None,None,None,1,0007998_0,direct_GE,31.0
11895,13765.0,7999,0,2022-01-01,20.0,30.0,8.0,,,,...,,,,--,--,,1,0007999_0,direct_GE,31.0


## Create point file for confusion matrices

In [7]:
pix = open_pix_info_sqlachemy(local_db_path)

pix_data = pd.merge(all_obs,pix,left_on='PIDi',right_on='PID',how='left')
## format to meet specs used in LUCinSA_helpers
pix_data.rename(columns={"PID0": "OID_", "cent_X": "Xcoord", "cent_Y":"Ycoord"})
pix_data['LC2'] = pix_data.apply(lambda x: 30 if x['LC'] == 30 else 0, axis=1)
pix_data.to_csv(os.path.join(out_dir,'samp2022_pix.csv'))

## Summarize sample size by sample group

In [9]:
pix = open_pix_info_sqlachemy(local_db_path)
#pix_data = pix_data[pix_data['PID1']==0]
pix_data = pd.merge(all_obs,pix,left_on='PIDi',right_on='PID',how='left')
samp_sum=pix_data.groupby(['sampgroup','LC']).size().reset_index(name='counts')
samp_tot=pix_data.groupby(['sampgroup']).size().reset_index(name='total')
samp = pd.merge(samp_sum,samp_tot,on='sampgroup',how='left')
samp['per'] = samp['counts']/samp['total']
samp[samp['LC']==30.0]

,sampgroup,LC,counts,total,per
0,D0102_Concepcion_B,30.0,68,494,0.137652
3,D0103_Concepcion_H,30.0,26,225,0.115556
6,D0112_Concepcion_A,30.0,15,169,0.088757
9,D0217_SanPedro_C,30.0,120,485,0.247423
12,D0301_Cordillera_C,30.0,19,399,0.047619
15,D0502_Caaguazu_C,30.0,99,403,0.245658
18,D0506_Caaguazu_DJMF,30.0,119,295,0.403390
21,D0518_Caaguazu_3F,30.0,160,490,0.326531
24,D0603_Caazapa_B,30.0,52,390,0.133333
27,D0607_Caazapa_SJN,30.0,43,328,0.131098
